# Obdelava podatkov

In [25]:
import re
from bs4 import BeautifulSoup
import requests
import os
import csv

## 1. Pridobivanje podatkov
Prvo definiramo spremenljivke:

In [26]:
podatki_url = 'https://en.wikipedia.org/wiki/List_of_chemical_elements'

directory_dat = 'Surovi podatki'

datoteka_html = 'podatki.html'


Potem napišemo funkcijo, ki podatke dobi iz spleta:

In [27]:
def nalozi_iz_spleta(url):
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            raise ValueError(f'Prišlo je do čudne kode: {response.status_code}')
    except Exception:
        print(f'Prišlo je do napake pri nalaganju strani {url}')

potem napisemo funkcijo, ki bo te podatke shranila v file:

In [28]:
def shrani_v_datoteko(text, directory, filename):
    if directory != '':
        os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    with open(path, 'w', encoding='utf-8') as file_out:
        file_out.write(text)
    return None

in še funkcije, ki združi prejšnji funkciji:

In [29]:
def shrani(page, directory, filename):
    vsebina = nalozi_iz_spleta(page)
    if vsebina != None:
        shrani_v_datoteko(vsebina, directory, filename) 

In [30]:
shrani(podatki_url, directory_dat, datoteka_html)

Prišlo je do napake pri nalaganju strani https://en.wikipedia.org/wiki/List_of_chemical_elements


## 2. Izluščenje podatkov

In [32]:
def luscenje(dat):
    juha = BeautifulSoup(datoteka_html, 'html.parser')
    tabela = juha.find('table', class_='wikitable sortable')
    vrstice = tabela.find_all('tr')
    for vr in vrstice[1:]:
        celice = vr.find_all('td')
        at_st = celice[0].get_text()
        sy = celice[1].get_text()
        name = celice[2].get_text()
        org = celice[3].get_text()
        group = celice[4].get_text()
        period = celice[5].get_text()
        block = celice[6].get_text()
        saw = celice[7].get_text()
        dens = celice[8].get_text()
        mlt_p = celice[9].get_text()
        boil_p = celice[10].get_text()
        shc = celice[11].get_text()
        el_neg = celice[12].get_text()
        abound = celice[13].get_text()
        orig = celice[14].get_text()
        phase = celice[15].get_text()
        print(saw, el_neg, orig)    

In [ ]:
print()